### They inner atoms, and the outer atoms are equally spaced

In [34]:
from qutip import *
import numpy as np
import os
import matplotlib.pyplot as plt
#from tqdm import tqdm
#from scipy.optimize import minimize
#import matplotlib.animation as animation
#from functools import partial

In [35]:
save_path = r'C:\Users\leopo\OneDrive - UT Cloud\Uni\Semester_8\BA_mit_Git\BA_Plots\Qutip'
os.makedirs(os.path.dirname(save_path), exist_ok=True)

In [36]:
fixed_lam = 1
fixed_gamma = 1

fixed_dist_ext = 0.235  * fixed_lam
fixed_dist_in  =  fixed_dist_ext
########################################                 Define constants                   #############################################
N = 3 + 3 * 3

In [37]:
########################################               define the geometry                 #############################################
def phi(t, tau):
    pi_start = np.pi
    pi_end = np.pi / 3
    
    # Define the transition intervals
    t1 = t_max / 10
    t2 = t1 + tau
    
    if t <= t1:
        return pi_start
    elif t >= t2:
        return pi_end
    else:
        # Normalize time to the range [0, pi] for the transition
        normalized_time = ((t - t1) / (t2 - t1)) * np.pi
        
        # Apply sin^2 transition
        sin_squared = np.sin(normalized_time / 2)**2
        
        return pi_start + (pi_end - pi_start) * sin_squared

def dipole_vector(t, tau):
    """Returns the dipole vector given its orientation by angle phi."""
    dipole = np.array([np.pi/2 + np.cos(phi(t, tau)), np.pi/2 + np.sin(phi(t, tau)), 0])
    return dipole

def z_rotation(angle):
    return np.array([
        [np.cos(angle), -np.sin(angle), 0],
        [np.sin(angle), np.cos(angle), 0],
        [0, 0, 1]])

def chain_positions(distance, N):
    Pos = np.zeros((N, 3))
    for i in range(N):
        Pos[i, 0] = i * distance
    return Pos

def topo_positions(distance_in = fixed_dist_in, distance_ext = fixed_dist_ext, N = N):
    Pos = np.zeros((N, 3))
    r = distance_in / np.sqrt(3)
    Chain = chain_positions(distance_ext, N // 3) + [r, 0, 0]
    Pos[:N//3] = np.dot(Chain, z_rotation(np.pi).T)
    Pos = Pos[Pos[:, 0].argsort()]
    Pos[N//3:2*N//3] = np.dot(Chain, z_rotation(2 * np.pi / 6).T)
    Pos[2*N//3:] = np.dot(Chain, z_rotation(-2 * np.pi / 6).T)
    return Pos

In [38]:
########################################             create the Hamiltonean                 #################################################
def Green_tensor(r_a, r_b, k_a):
    r_ab = r_b - r_a
    abs_r_ab = np.linalg.norm(r_ab)
    kappa = k_a * abs_r_ab
    return (np.exp(1j * kappa) / (4 * np.pi * kappa ** 2 * abs_r_ab)
                    * ((kappa ** 2 + 1j * kappa - 1) * np.eye(3)
                       + (- kappa ** 2 - 3 * 1j * kappa + 3)
                       * np.outer(r_ab, r_ab) / (abs_r_ab ** 2)))

def Gamma_matrix(distance_in, distance_ext, dipoles, lam, gamma):
    positions = topo_positions(distance_in, distance_ext)
    G_matrix = np.zeros((N, N), dtype=complex)
    for a in range(N):
        for b in range(N):
            G_matrix[a, b] = gamma
            r_a, r_b = positions[a], positions[b]
            if np.linalg.norm(r_b - r_a) > 1e-5:
                d_a, d_b = dipoles[a], dipoles[b]
                k_a = 2 * np.pi / lam
                G_matrix[a, b] = np.imag((6 * np.pi * gamma / k_a * np.matmul(np.conj(d_a), np.matmul(Green_tensor(r_a, r_b, k_a), d_b.T))))
    return G_matrix

def V_matrix(distance_in, distance_ext, dipoles, lam, gamma):
    positions = topo_positions(distance_in, distance_ext)
    V_matrix = np.zeros((N, N), dtype=complex)
    for a in range(N):
        for b in range(N):
            r_a, r_b = positions[a], positions[b]
            V_matrix[a, b] = 0
            if np.linalg.norm(r_b - r_a) > 1e-5:
                d_a, d_b = dipoles[a], dipoles[b]
                k_a = 2 * np.pi / lam
                V_matrix[a, b] = np.real((3 * np.pi * gamma / k_a * np.matmul(np.conj(d_a), np.matmul(Green_tensor(r_a, r_b, k_a), d_b.T))))
    return V_matrix
    
def H_eff(distance_in = fixed_dist_in, distance_ext = fixed_dist_ext, t=0, tau=0, lam=fixed_lam, gamma=fixed_gamma):
    dipoles = [dipole_vector(t, tau) for _ in range(N)]
    G = Gamma_matrix(distance_in, distance_ext, dipoles, lam, gamma)
    V = V_matrix(distance_in, distance_ext, dipoles, lam, gamma)
    return Qobj(V) - 1j / 2 * Qobj(G)

In [39]:
################ initiate Wave-packet
k_s = np.pi / fixed_dist_in
sigma = 0.1 * k_s

def create_wave_packet(N):
    center_index = N // 6
    x_j = (np.arange(N//3) - center_index) * fixed_dist_ext
    coefficients = np.sqrt(sigma / np.sqrt(2 * np.pi)) * np.exp(-1j * k_s * x_j) * np.exp(-sigma**2 * x_j**2)
    wave_packet = sum(coeff * basis(N, j) for j, coeff in enumerate(coefficients))
    return wave_packet.unit()

In [ ]:
t_max    = 100
t_size   = 100
########################################################################################################################    
times  = np.linspace(0, t_max, t_size) * fixed_gamma
# Define the arguments for the Hamiltonian
tau = 0.01 * t_max
args = {'tau': tau}

# Define solver options with increased nsteps
opts = Options(nsteps=10000)

# Solve the time evolution
result = mesolve(H_eff, create_wave_packet(N), times, [], [], args=args, options=opts)

# Extract and plot the results
populations = [expect(basis(N, i).proj(), result.states) for i in range(N)]

# Plot the time evolution of atom state probabilities as a heatmap
plt.figure(figsize=(10, 6))
plt.imshow(populations, aspect='auto', origin='lower', cmap='viridis', extent=[0, times[-1], 0, N-1])
plt.colorbar(label='Probability')
plt.xlabel('Time')
plt.ylabel('Atom index')
plt.title('Time evolution of atom state probabilities')
plt.show()

C:\Users\leopo\anaconda3\Lib\site-packages\qutip\solver\options.py:16: FutureWarning: Dedicated options class are no longer needed, options should be passed as dict to solvers.
  warnings.warn(
